# What's the state of indoor agriculture in the US?

Having worked in food distribution for a decade, I probably spend more time thinking about the logistics of food than most people do. When I eat, it's hard for me *not to* imagine where my food came from and how it reached me. Still, it wasn't until the pandemic approached New York City that the thought of losing access to fresh food began to seem like a real possiblity. The combination of fear and free time is powerful. 

I discovered my long dormant interest in gardening had reawakened, seemingly overnight. The ability to grow a few vegetables at home did provide a little bit of comfort. It was really the knowledge, though, that food was being grown on a commercial scale within walking distance of my home that was able to nudge my mind closer to ease. For years, I'd happily been paying a premium for hydroponically grown greens based on their quality alone. But suddenly, as the pandemic gained a foothold and the city locked down, what had once been to me a source of luxury - indoor farms - became a symbol of hope. 

Sure the world was a mess, but we might just be able to dig ourselves out of it.

## Population density and the emergence of indoor farming

Once I became attuned, it seemed indoor farms began cropping up left and right. Apparently, the industry was quite a bit larger than I'd realized. Looking to understand the landscape better, the question of *where* exactly these farms were located seemed like a good starting point.

What kinds of places become home to indoor farms? There are a lot of ways to characterize a place, so I decided to narrow my initial scope to population density using data from the most recent US Census, 2020. My own neighborhood can very significantly in the span of a block or two, so I wanted to work with relatively granular population data. While census *blocks* are actually the smallest geographic unit for which census data is gathered, at the time I began this project, the 2020 data at this level had not yet been released. Data at the census tract level, however, was available and I decided it would suffice. 

## Taming the data beast

The [US Decennial Census of Population and Housing](https://www.census.gov/programs-surveys/decennial-census/data/datasets.html) provides a very rich source of raw data, but it can be challenging to decipher and synthesize into usable features. This set of project notebooks walks through the data cleaning and analysis techniques I applied towards my goal of contributing to the understanding of the indoor farming industry and its role in overall food security. The "Data Sources" section of each notebook details the individual files used and the specific steps taken to access them. 

In [1]:
# the code below uses the `os` and `pathlib` modules to make it easier to 
# refer to the various folders where data files for this project are stored.

import os, pathlib
base_dir = pathlib.Path(os.getcwd()).parent
data_archive_dir = os.path.join(base_dir, "data_archive")
clean_data_dir = os.path.join(data_archive_dir, "clean")
data_dir = os.path.join(base_dir, "data")
shapes_dir = os.path.join(data_dir,"shapes")
json_dir = os.path.join(data_dir,"geojson")
util_dir = os.path.join(data_dir,"util")

## *The 50,000 foot view: NYC population density*

In [2]:
import pandas as pd

In [3]:
# load GEO DataFrame of the "place" New York City from CSV file
clean_geodf_place_nyc_file_csv  = os.path.join(clean_data_dir, "geodf_place_nyc.csv")
geodf_place_nyc = pd.read_csv(clean_geodf_place_nyc_file_csv, index_col=0)
geodf_place_nyc.head()
#= pa.Table.from_pandas(ny_shapes_df, preserve_index=False)

,State,State FIPS,Place,Place FIPS,County,Land Area,geometry
State Name,,,,,,,
New York,NY,36,New York City,51000,"Bronx County, Kings County, New York County, Q...",778165283,"MULTIPOLYGON (((-74.040747 40.700167, -74.0407..."


In [4]:
# load GEO DataFrame of NY State census tracts from CSV file
clean_geodf_tract_ny_file_csv = os.path.join(clean_data_dir,'tl_2020_36_tract20.csv')
geodf_tract_ny = pd.read_csv(clean_geodf_tract_ny_file_csv, index_col=0)
geodf_tract_ny.head(3)

,County FIPS,Census Tract Name,GEOID Census Tract,Population,Land Area,Population Density,geometry
State FIPS,,,,,,,
36,47,"Census Tract 7, Kings County, New York",36047000700,4415,176774,64686,"POLYGON ((-74.001542 40.692786, -74.001317 40...."
36,47,"Census Tract 9, Kings County, New York",36047000900,5167,163469,81865,"POLYGON ((-73.994046 40.690897, -73.993736 40...."
36,47,"Census Tract 11, Kings County, New York",36047001100,1578,168507,24254,"POLYGON ((-73.99072799999999 40.693048, -73.99..."


In [5]:
# # load GEO DataFrame of NY State places from CSV file
# clean_geodf_place_ny_file_csv = os.path.join(clean_data_dir,'geodf_place_ny.csv')
# geodf_place_ny = pd.read_csv(clean_geodf_place_ny_file_csv, index_col=0)
# geodf_place_ny.head(3)

In [6]:
from keplergl import KeplerGl
ny_map = KeplerGl(height=600, show_docs=False)
ny_map.add_data(data=geodf_place_nyc, name='New York City') 
ny_map.add_data(data=geodf_tract_ny, name='Population Density')

# geodf_place_nyc["geometry"].explode().iloc[7]
# geodf_place_nyc["geometry"].boundary.plot()


In [7]:
ny_map.config = {'version': 'v1',
    'config': {'visState': {'filters': [],
      'layers': [{'id': 'm7mkzo',
        'type': 'geojson',
        'config': {'dataId': 'New York City',
         'label': 'New York City',
         'color': [183, 136, 94],
         'highlightColor': [252, 242, 26, 255],
         'columns': {'geojson': 'geometry'},
         'isVisible': True,
         'visConfig': {'opacity': 0.8,
          'strokeOpacity': 0.8,
          'thickness': 1.5,
          'strokeColor': [18, 147, 154],
          'colorRange': {'name': 'Global Warming',
           'type': 'sequential',
           'category': 'Uber',
           'colors': ['#5A1846',
            '#900C3F',
            '#C70039',
            '#E3611C',
            '#F1920E',
            '#FFC300']},
          'strokeColorRange': {'name': 'Global Warming',
           'type': 'sequential',
           'category': 'Uber',
           'colors': ['#5A1846',
            '#900C3F',
            '#C70039',
            '#E3611C',
            '#F1920E',
            '#FFC300']},
          'radius': 10,
          'sizeRange': [0, 10],
          'radiusRange': [0, 50],
          'heightRange': [0, 500],
          'elevationScale': 5,
          'enableElevationZoomFactor': True,
          'stroked': True,
          'filled': False,
          'enable3d': False,
          'wireframe': False},
         'hidden': False,
         'textLabel': [{'field': None,
           'color': [255, 255, 255],
           'size': 18,
           'offset': [0, 0],
           'anchor': 'start',
           'alignment': 'center'}]},
        'visualChannels': {'colorField': None,
         'colorScale': 'quantile',
         'strokeColorField': None,
         'strokeColorScale': 'quantile',
         'sizeField': None,
         'sizeScale': 'linear',
         'heightField': None,
         'heightScale': 'linear',
         'radiusField': None,
         'radiusScale': 'linear'}},
       {'id': 'vqgaiv',
        'type': 'geojson',
        'config': {'dataId': 'Population Density',
         'label': 'Population Density',
         'color': [248, 149, 112],
         'highlightColor': [252, 242, 26, 255],
         'columns': {'geojson': 'geometry'},
         'isVisible': True,
         'visConfig': {'opacity': 0.8,
          'strokeOpacity': 0.8,
          'thickness': 0.5,
          'strokeColor': [130, 154, 227],
          'colorRange': {'name': 'Global Warming 8',
           'type': 'sequential',
           'category': 'Uber',
           'colors': ['#4C0035',
            '#650031',
            '#7F0023',
            '#98000A',
            '#B21800',
            '#CB4600',
            '#E57F00',
            '#FFC300']},
          'strokeColorRange': {'name': 'Global Warming',
           'type': 'sequential',
           'category': 'Uber',
           'colors': ['#5A1846',
            '#900C3F',
            '#C70039',
            '#E3611C',
            '#F1920E',
            '#FFC300']},
          'radius': 10,
          'sizeRange': [0, 10],
          'radiusRange': [0, 50],
          'heightRange': [0, 500],
          'elevationScale': 5,
          'enableElevationZoomFactor': True,
          'stroked': False,
          'filled': True,
          'enable3d': False,
          'wireframe': False},
         'hidden': False,
         'textLabel': [{'field': None,
           'color': [255, 255, 255],
           'size': 18,
           'offset': [0, 0],
           'anchor': 'start',
           'alignment': 'center'}]},
        'visualChannels': {'colorField': {'name': 'Population',
          'type': 'integer'},
         'colorScale': 'quantile',
         'strokeColorField': None,
         'strokeColorScale': 'quantile',
         'sizeField': None,
         'sizeScale': 'linear',
         'heightField': None,
         'heightScale': 'linear',
         'radiusField': None,
         'radiusScale': 'linear'}},
       {'id': 'g9rm7n',
        'type': 'geojson',
        'config': {'dataId': 'NY State Places',
         'label': 'NY State Places',
         'color': [231, 159, 213],
         'highlightColor': [252, 242, 26, 255],
         'columns': {'geojson': 'geometry'},
         'isVisible': False,
         'visConfig': {'opacity': 0.8,
          'strokeOpacity': 0.8,
          'thickness': 0.5,
          'strokeColor': [30, 150, 190],
          'colorRange': {'name': 'Global Warming',
           'type': 'sequential',
           'category': 'Uber',
           'colors': ['#5A1846',
            '#900C3F',
            '#C70039',
            '#E3611C',
            '#F1920E',
            '#FFC300']},
          'strokeColorRange': {'name': 'Global Warming',
           'type': 'sequential',
           'category': 'Uber',
           'colors': ['#5A1846',
            '#900C3F',
            '#C70039',
            '#E3611C',
            '#F1920E',
            '#FFC300']},
          'radius': 10,
          'sizeRange': [0, 10],
          'radiusRange': [0, 50],
          'heightRange': [0, 500],
          'elevationScale': 5,
          'enableElevationZoomFactor': True,
          'stroked': True,
          'filled': True,
          'enable3d': False,
          'wireframe': False},
         'hidden': False,
         'textLabel': [{'field': None,
           'color': [255, 255, 255],
           'size': 18,
           'offset': [0, 0],
           'anchor': 'start',
           'alignment': 'center'}]},
        'visualChannels': {'colorField': None,
         'colorScale': 'quantile',
         'strokeColorField': None,
         'strokeColorScale': 'quantile',
         'sizeField': None,
         'sizeScale': 'linear',
         'heightField': None,
         'heightScale': 'linear',
         'radiusField': None,
         'radiusScale': 'linear'}}],
      'interactionConfig': {'tooltip': {'fieldsToShow': {'New York City': [{'name': 'State',
           'format': None},
          {'name': 'State FIPS', 'format': None},
          {'name': 'Place', 'format': None},
          {'name': 'Place FIPS', 'format': None},
          {'name': 'County', 'format': None}],
         'NY State': [{'name': 'County FIPS', 'format': None},
          {'name': 'Census Tract Name', 'format': None},
          {'name': 'GEOID Census Tract', 'format': None},
          {'name': 'Population', 'format': None},
          {'name': 'Land Area', 'format': None}],
         'NY State Places': [{'name': 'State', 'format': None},
          {'name': 'State FIPS', 'format': None},
          {'name': 'Place', 'format': None},
          {'name': 'Place FIPS', 'format': None},
          {'name': 'County', 'format': None}]},
        'compareMode': False,
        'compareType': 'absolute',
        'enabled': True},
       'brush': {'size': 0.5, 'enabled': False},
       'geocoder': {'enabled': False},
       'coordinate': {'enabled': False}},
      'layerBlending': 'normal',
      'splitMaps': [],
      'animationConfig': {'currentTime': None, 'speed': 1}},
     'mapState': {'bearing': 0,
      'dragRotate': False,
      'latitude': 40.72146066254021,
      'longitude': -73.65313270315059,
      'pitch': 0,
      'zoom': 9,
      'isSplit': False},
     'mapStyle': {'styleType': 'dark',
      'topLayerGroups': {},
      'visibleLayerGroups': {'label': True,
       'road': True,
       'border': False,
       'building': True,
       'water': True,
       'land': True,
       '3d building': False},
      'threeDBuildingColor': [9.665468314072013,
       17.18305478057247,
       31.1442867897876],
      'mapStyles': {}}}}

In [8]:
ny_map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'm7mkzo', 'type': '…

## *Up close*
The following project notebooks provide a detailed look at the various steps of cleaning and mapping the data for this project. The final step of mapping indoor farms in New York City will follow sh
* [Data Cleanup: US Census population data](http://localhost:8888/lab/tree/notebooks/01_DataCleanup_Population.ipynb)
* [Data Cleanup: GEOID tables](http://localhost:8888/lab/tree/notebooks/02_DataCleanup_GEOIDs.ipynb)
* [Data Cleanup: Geographic data](http://localhost:8888/lab/tree/notebooks/03_DataCleanup_GIS.ipynb)
* [Mapping NY State: Population Density](http://localhost:8888/lab/tree/notebooks/04_Mapping_Population.ipynb)
* Mapping New York City: Indoor Farms (coming soon...)